# Notebook for plotting results of slice histology
Useful for looking at the expression of (sparse/cre-dependent/not) opsin 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from czifile import CziFile

from skimage.filters import threshold_otsu
from skimage.filters import gaussian
from skimage.transform import rotate


In [ ]:
path = '/Users/jure/Desktop/IP-ApoTome-05.czi'
clip_rgb=[99.5, 99.0, 99.0]

In [ ]:
def plot_steps(img_sum, img_bin, img_diff, thresh, fit_line, slope, angle):
    # visualise the pipeline
    fig, axs = plt.subplots(2, 2, figsize=(8, 8), dpi=300)
    axs[0, 0].imshow(img_sum, cmap='gray')
    axs[0, 0].set_title('Smoothed sum of channels')

    axs[0, 1].hist(img_sum.ravel(), bins=20, color='C0')
    axs[0, 1].axvline(thresh, color='C1', linestyle='--')
    axs[0, 1].set_title('Histogram + Otsu thr.')

    axs[1, 0].imshow(img_bin, cmap='gray')
    axs[1, 0].set_title('Binarised image')

    axs[1, 1].imshow(img_diff, cmap='gray')
    axs[1, 1].plot(np.arange(img_diff.shape[1]), fit_line, color='C2', label=f'Slope: {slope:.2f}\nAngle: {angle:.2f}°')
    axs[1, 1].set_ylim(img_diff.shape[0], 0)
    axs[1, 1].set_xlim(0, img_diff.shape[1])
    axs[1, 1].set_title('Diff image + fit line')
    axs[1, 1].legend()

    for ax in axs.flat:
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
# generate basic function to visualise apotome/confocal image given a path to a .czi file
def preproc_czi(path, clip_rgb= [99.5, 99.5, 99.5], smooth_sigma=10):

    # 1) load image and convert to (x, y, c)
    with CziFile(path) as czi:
        img = np.array(czi.asarray())

    img = img.squeeze().transpose((1, 2, 0))  # reorder dimensions if necessary
    img = img[:, :, [2, 1, 0]]  # assuming the image is in RGB format

    img_proc = np.zeros_like(img, dtype=np.uint8)

    # 2) preprocess by clipping and normalising each channel
    for i in range(3):
        img_ch = img[..., i]
        img_ch = img_ch / img_ch.max()
        img_ch = np.clip(img_ch, 0, np.percentile(img_ch, clip_rgb[i]))
        img_ch = img_ch / img_ch.max()
        img_ch = (img_ch * 255).astype(np.uint8)
        img_proc[..., i] = img_ch

    # 3) detect the surface and compute angle by fitting a line to the points

    # 3.1) sum the channels and smooth
    img_sum = img_proc.sum(axis=-1)
    img_sum = gaussian(img_sum, sigma=smooth_sigma)

    # 3.2) binarise using otsu's method
    thresh = threshold_otsu(img_sum)
    img_bin = img_sum > thresh

    # 3.3) detect the x and y coordinates of edges
    img_diff_x = np.diff(img_bin, axis=1)[:-1, :]  # remove last column
    img_diff_y = np.diff(img_bin, axis=0)[:, :-1]  # remove last row
    img_diff = np.sqrt(img_diff_x**2 + img_diff_y**2)

    y, x = np.where(img_diff > 0)

    # 3.4) fit a line to the points
    fit = np.polyfit(x, y, 1)  # fit a line to the points
    fit_line = np.polyval(fit, np.arange(img_diff.shape[1]))
    slope = fit[0]
    angle = np.arctan(slope) * 180 / np.pi  # convert to degrees

    # 3.5) plot the steps of point 3)
    plot_steps(img_sum, img_bin, img_diff, thresh, fit_line, slope, angle)
    
    # 4) Rotate the image by the angle
    img_rotated = rotate(img_proc, angle + 180, resize=False)

    return img_rotated  # return the rotated image for further processing if needed
    

In [ ]:
img = preproc_czi(path, clip_rgb=clip_rgb)  # adjust clip values as needed

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(img)
plt.axis('off')
plt.show()
